In [91]:
import pandas as pd

# Load the dataset
df = pd.read_csv('raw.csv')  # Change to the path of your CSV file

# Data Preparation
# For data till the end of Mar 2024, you should input 2024, 3 as the year and month 
year, month = 2024, 3

# Convert 'MemberEvent.Date' to datetime
df['MemberEvent.Date'] = pd.to_datetime(df['MemberEvent.Date'])

# Correctly handle the transition from December to January of the next year
if month == 12:
    next_month_year = year + 1
    next_month = 1
else:
    next_month_year = year
    next_month = month + 1

# Use the next month's first day as the cutoff point for filtering
cutoff_date = pd.Timestamp(year=next_month_year, month=next_month, day=1, tz='UTC')

# Adjust filtering to exclude entries from the specified cutoff date onwards
filtered_df = df[df['MemberEvent.Date'] < cutoff_date]

# Calculation 1: First 5 Token Earners
latest_entries = filtered_df.sort_values(by='MemberEvent.Date', ascending=False).drop_duplicates(subset='MemberEvent.Member.Person.DisplayName')
sorted_latest_entries = latest_entries.sort_values(by='MemberEvent.MetaData.points-after-change', ascending=False).head(5)
print("First 5 Token Holders (Latest Entry Per Member):")
print(sorted_latest_entries[['MemberEvent.Member.Person.DisplayName', 'MemberEvent.MetaData.points-after-change']])


# Further filter the dataset to only include records from the specific year and month for calculations 2 and 3
month_specific_df = filtered_df[
    (filtered_df['MemberEvent.Date'].dt.year == year) & 
    (filtered_df['MemberEvent.Date'].dt.month == month)
]

# Calculation 2: Top 5 Total Token Earners of the Month (Calculate positive earnings only)
# Filter out negative earnings
positive_earnings_df = month_specific_df[month_specific_df['MemberEvent.MetaData.points-after-change'] > month_specific_df['MemberEvent.MetaData.points-before-change']]
# Calculate net positive earnings
positive_earnings_df['Net Positive Earnings'] = positive_earnings_df['MemberEvent.MetaData.points-after-change'] - positive_earnings_df['MemberEvent.MetaData.points-before-change']
total_positive_earnings = positive_earnings_df.groupby('MemberEvent.Member.Person.DisplayName')['Net Positive Earnings'].sum().reset_index()
top_5_positive_earnings = total_positive_earnings.sort_values(by='Net Positive Earnings', ascending=False).head(5)
print("\nTop 5 Total Positive Token Earners of the Specified Month:")
print(top_5_positive_earnings)

# Calculation 3: Total Tokens Issued, Spent, and Net within the specified month
month_specific_df['Net Earnings'] = month_specific_df['MemberEvent.MetaData.points-after-change'] - month_specific_df['MemberEvent.MetaData.points-before-change']
tokens_issued_month = month_specific_df[month_specific_df['Net Earnings'] > 0]['Net Earnings'].sum()
tokens_spent_month = month_specific_df[month_specific_df['Net Earnings'] < 0]['Net Earnings'].sum()
net_tokens_month = tokens_issued_month + tokens_spent_month
print("\nTotal Tokens Issued, Spent, and Net of the Specified Month:")
print(f"Tokens Issued: {tokens_issued_month}")
print(f"Tokens Spent: {tokens_spent_month}")
print(f"Net Tokens: {net_tokens_month}")

First 5 Token Holders (Latest Entry Per Member):
    MemberEvent.Member.Person.DisplayName  \
138                          Christie Lai   
16                             Davis Chow   
26                                Ryan Ng   
22                              Travis Ye   
43                            Calvin Kwan   

     MemberEvent.MetaData.points-after-change  
138                                    4160.0  
16                                     3350.0  
26                                     2170.0  
22                                     2160.0  
43                                     1600.0  

Top 5 Total Positive Token Earners of the Specified Month:
   MemberEvent.Member.Person.DisplayName  Net Positive Earnings
11                          Christie Lai                 4160.0
57                               Ryan Ng                 1610.0
64                         Vincent Cheng                  990.0
8                            Calvin Kwan                  790.0
31          

/var/folders/xj/9f51401j40b1p9mf8kpsb8s80000gn/T/ipykernel_44612/3734337956.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_earnings_df['Net Positive Earnings'] = positive_earnings_df['MemberEvent.MetaData.points-after-change'] - positive_earnings_df['MemberEvent.MetaData.points-before-change']
/var/folders/xj/9f51401j40b1p9mf8kpsb8s80000gn/T/ipykernel_44612/3734337956.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month_specific_df['Net Earnings'] = month_specific_df['MemberEvent.MetaDa

In [94]:
import pandas as pd

def load_and_prepare_data(filepath, year, month):
    df = pd.read_csv(filepath)
    df['MemberEvent.Date'] = pd.to_datetime(df['MemberEvent.Date'])
    
    # Calculate cutoff date for filtering
    cutoff_date = pd.Timestamp(year=year, month=month, day=1, tz='UTC') + pd.DateOffset(months=1)
    filtered_df = df[df['MemberEvent.Date'] < cutoff_date]
    return filtered_df

def get_first_5_token_holders(df):
    latest_entries = df.sort_values(by='MemberEvent.Date', ascending=False).drop_duplicates(subset='MemberEvent.Member.Person.DisplayName')
    top_5_holders = latest_entries.sort_values(by='MemberEvent.MetaData.points-after-change', ascending=False).head(5)
    return top_5_holders[['MemberEvent.Member.Person.DisplayName', 'MemberEvent.MetaData.points-after-change']]

def calculate_positive_earnings(df, year, month):
    positive_earnings_df = df.copy()
    positive_earnings_df['Net Positive Earnings'] = positive_earnings_df['MemberEvent.MetaData.points-after-change'] - positive_earnings_df['MemberEvent.MetaData.points-before-change']
    positive_earnings_df = positive_earnings_df[(positive_earnings_df['Net Positive Earnings'] > 0) & 
                                                 (positive_earnings_df['MemberEvent.Date'].dt.year == year) & 
                                                 (positive_earnings_df['MemberEvent.Date'].dt.month == month)]
    total_positive_earnings = positive_earnings_df.groupby('MemberEvent.Member.Person.DisplayName')['Net Positive Earnings'].sum().reset_index()
    top_5_positive_earnings = total_positive_earnings.sort_values(by='Net Positive Earnings', ascending=False).head(5)
    return top_5_positive_earnings

def calculate_total_tokens(df, year, month):
    df['Net Earnings'] = df['MemberEvent.MetaData.points-after-change'] - df['MemberEvent.MetaData.points-before-change']
    month_specific_df = df[(df['MemberEvent.Date'].dt.year == year) & (df['MemberEvent.Date'].dt.month == month)]
    tokens_issued_month = month_specific_df[month_specific_df['Net Earnings'] > 0]['Net Earnings'].sum()
    tokens_spent_month = month_specific_df[month_specific_df['Net Earnings'] < 0]['Net Earnings'].sum()
    return tokens_issued_month, tokens_spent_month, tokens_issued_month + tokens_spent_month

# Main processing function
def process_data(filepath, year, month):
    df = load_and_prepare_data(filepath, year, month)
    print("First 5 Token Holders (Latest Entry Per Member):")
    print(get_first_5_token_holders(df))
    print("\nTop 5 Total Positive Token Earners of the Specified Month:")
    print(calculate_positive_earnings(df, year, month))
    tokens_issued, tokens_spent, net_tokens = calculate_total_tokens(df, year, month)
    print("\nTotal Tokens Issued, Spent, and Net of the Specified Month:")
    print(f"Tokens Issued: {tokens_issued}")
    print(f"Tokens Spent: {tokens_spent}")
    print(f"Net Tokens: {net_tokens}")

# Example usage
process_data('raw.csv', 2024, 3)


First 5 Token Holders (Latest Entry Per Member):
    MemberEvent.Member.Person.DisplayName  \
138                          Christie Lai   
16                             Davis Chow   
26                                Ryan Ng   
22                              Travis Ye   
43                            Calvin Kwan   

     MemberEvent.MetaData.points-after-change  
138                                    4160.0  
16                                     3350.0  
26                                     2170.0  
22                                     2160.0  
43                                     1600.0  

Top 5 Total Positive Token Earners of the Specified Month:
   MemberEvent.Member.Person.DisplayName  Net Positive Earnings
11                          Christie Lai                 4160.0
57                               Ryan Ng                 1610.0
64                         Vincent Cheng                  990.0
8                            Calvin Kwan                  790.0
31          